# Simulating Single Cell Data for Pathway Analysis
This notebook simulates single cell RNA-seq data with differentially expressed pathways across cell populations. The workflow includes:
- Loading gene sets and pathway definitions
- Filtering and mapping gene names
- Selecting pathways with high-intensity genes
- Assigning pathways to experimental states
- Simulating gene expression data with SPARSim
- Analyzing simulated data with Seurat
- Saving results for downstream benchmarking

---
Author: Felix Offensperger

In [ ]:
# Load required R libraries for simulation and analysis
library(SPARSim)      # For simulating single cell RNA-seq data
library(dplyr)        # Data manipulation
library(Seurat)       # Single cell analysis
library(qusage)       # Gene set/pathway analysis
library(stringr)      # String operations
library(ggplot2)      # Plotting
library('biomaRt')    # Accessing BioMart for gene annotation

# Show session information for reproducibility
sessionInfo()

In [ ]:
# Install required Bioconductor packages (if not already installed)
BiocManager::install(c("qusage", "edgeR", "mclust", "scater", "scran"))

In [ ]:
# Load pathway definitions from a GMT file
define_file = "go_human.bp.gmt"
pathways <- qusage::read.gmt(define_file)



In [ ]:
# Find pathway names containing the word 'receptor'
grep("receptor", names(pathways), value=T)

In [ ]:
# Load preset simulation parameters for PBMC 10X dataset
# (Other presets are listed in the commented line)
data("PBMC_10X_param_preset")
intensity = PBMC_10X_param_preset[[1]]$intensity

# Create a data frame of gene names (Ensembl IDs)
genes_df = data.frame(names=names(intensity))

# Map Ensembl gene IDs to HGNC symbols using biomaRt
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))
G_list <- getBM(filters= "ensembl_gene_id", attributes= c("ensembl_gene_id","hgnc_symbol"),values=genes_df$name,mart= mart)

genes_df = merge(genes_df,G_list,by.x="names",by.y="ensembl_gene_id",all.x=TRUE)

# Remove duplicated gene symbols and assign unique placeholders for missing symbols
genes_df$hgnc_symbol[duplicated(genes_df$hgnc_symbol)] <- NA
genes_df[is.na(genes_df$hgnc_symbol),]$hgnc_symbol=paste0("NA_",1:sum(is.na(genes_df$hgnc_symbol)))

genes = genes_df$hgnc_symbol
names(intensity) = genes

# Keep only genes with valid symbols (not NA_*)
keepIndices = grep("^NA_", names(intensity), invert=T)
intensity = intensity[ keepIndices ]
variability = PBMC_10X_param_preset[[1]]$variability[ keepIndices ]

# Set up simulation parameters
genes = names(intensity)
n_genes = length(genes)
library_size = PBMC_10X_param_preset[[1]]$lib_size  * 10 # scale library size
n_cells = length(library_size)
genes = names(intensity)

In [ ]:
# Visualize the distribution of gene intensity values
hist(intensity, xlim = c(0, 3), breaks=2000)

In [ ]:
# Identify genes with high intensity (expression > 0.05)
highIntensityGenes = names(intensity[intensity > 0.05])

In [ ]:
# Select pathways with a high proportion of high-intensity genes
keepPWs = c()

for (pw in names(pathways))
{
    numGenes = length(pathways[[pw]])
    numHighIntensityGenes = length(intersect(highIntensityGenes, pathways[[pw]]))

    highIntensityRatio = numHighIntensityGenes / numGenes

    # Keep pathways with >1 gene and >50% high-intensity genes
    if ((numGenes > 1) && (highIntensityRatio > 0.5))
    {
        print(paste(pw, numGenes, numHighIntensityGenes))
        keepPWs = c(keepPWs, pw)
    }
}

# Assign selected pathways to experimental states, ensuring minimal overlap
keepPWs[1:10]

# Set up state-to-pathway and state-to-gene mappings
states = c("wildtype","knockout01","knockout02","knockout03")
num_pws_per_state = 10
state2pw = list()
state2genes = list()
state2count = list()
for (state in states) { state2count[[state]] = 0 }

for (pw in keepPWs) {
    pwGenes = pathways[[pw]]
    pwGenes = intersect(pwGenes, names(intensity))
    intersections = list()
    for (state in states) {
        if (state %in% names(state2genes)) {
            commonGenes = intersect(pwGenes, state2genes[[state]])
            if (length(commonGenes) > 0) {
                intersections[[state]] = length(commonGenes)
            }
        }
    }
    curstate = NULL
    if (length(names(intersections)) == 1) {
        nextstate = names(intersections)[1]
        if (state2count[[nextstate]] < num_pws_per_state) {
            curstate = nextstate
        }
    } else if (length(names(intersections)) == 0) {
        for (state in states) {
            if (state2count[[state]] < num_pws_per_state) {
                curstate = state
                break
            }
        }
    }
    if (!is.null(curstate)) {
        if (curstate %in% names(state2genes)) {
            state2genes[[curstate]] = c(state2genes[[curstate]], pwGenes)
            state2pw[[curstate]] = c(state2pw[[curstate]], pw)
            state2count[[curstate]] = state2count[[curstate]] + 1
        } else {
            state2genes[[curstate]] = c(pwGenes)
            state2pw[[curstate]] = c(pw)
            state2count[[curstate]] = state2count[[curstate]] + 1
        }
    }
}

# Print the assigned pathways for each state
print(state2pw)

In [ ]:
# Visualize the log-transformed intensity distribution
hist(log(intensity+1))

# Set up scaling factors for pathway changes in each state
# Each pathway assigned to a state will have its gene intensities scaled accordingly
changes = list()
set.seed(1)

# Define trends for pathway changes across states
trends = list(c(1,1,4,4), c(1,8,8,1), c(2,2,1,1))

for(i in 1:length(trends)){
  statename = names(state2pw)[i]
  for (pw in state2pw[[statename]]) {
    changes[[pw]] = unlist(trends[i])
  }
}

# Create a data frame of scaling factors for each pathway and state
changes_df = data.frame(Reduce(rbind, changes))
rownames(changes_df) = names(changes)
colnames(changes_df) = states
write.table(changes_df, file = "simulated_changingPathways_random.tsv", sep="\t", row.names = TRUE, col.names=TRUE, quote = FALSE)

# Prepare simulation parameters for each state
parameter_list = lapply(states, function(x)
  list(intensity=intensity, variability=variability, library_size=library_size, feature_names=genes, sample_names=paste0(x,"-Cell",1:n_cells), condition_name=x)
)
names(parameter_list) = states

# Apply scaling factors to gene intensities for each pathway in each state
for(c in names(changes)){
  scaling = changes[[c]]
  geneset = intersect(pathways[[c]], names(intensity))
  for(i in 1:length(parameter_list)){
    name_i = names(parameter_list)[i]
    parameter_list[[i]]$intensity[names(intensity) %in% geneset] = parameter_list[[i]]$intensity[names(intensity) %in% geneset] * scaling[i]
  }
}

# Print summary of simulation parameters and show genes with changing intensities
print(summary(parameter_list))
intensity_matrix = do.call(cbind, lapply(parameter_list, function(x)x$intensity))
print("Changing genes:")
intensity_matrix[unlist(apply(intensity_matrix,1, sd, na.rm = TRUE))!=0, ]

In [ ]:
# Check dimensions of simulation parameters
# (Useful for debugging and ensuring consistency)
dim(intensity)
dim(parameter_list[[1]]$variability)
dim(parameter_list[[2]]$intensity)
dim(parameter_list[[3]]$intensity)
dim(parameter_list[[4]]$intensity)

In [ ]:
# Create simulation parameters for each condition using SPARSim
param_per_condition = lapply(parameter_list, function(x)
  SPARSim_create_simulation_parameter(
    intensity = x$intensity, 
    variability = x$variability, 
    library_size = x$library_size,
    feature_names = x$feature_names, 
    sample_names = x$sample_names, 
    condition_name = x$condition_name)
)

# Simulate count data for all conditions
sim_result <- SPARSim_simulation(dataset_parameter = param_per_condition)

# Extract condition labels from simulated data
condition_vector = sapply(stringr::str_split(colnames(sim_result$count_matrix),"-"), "[[", 1)

In [ ]:
# Prepare simulated count matrix for Seurat analysis
rownames(sim_result$count_matrix) = make.names(rownames(sim_result$count_matrix))
colnames(sim_result$count_matrix) = make.names(colnames(sim_result$count_matrix))

# Create Seurat object from simulated data
sObject <- CreateSeuratObject(counts = sim_result$count_matrix, project = "simulated", min.cells = 0, min.features = 200)

# Calculate mitochondrial gene percentage
sObject[["percent.mt"]] <- PercentageFeatureSet(sObject, pattern = "^MT-")

# Visualize QC metrics
VlnPlot(sObject, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
plot1 <- FeatureScatter(sObject, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(sObject, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2

# Normalize and identify variable features
sObject <- NormalizeData(sObject, normalization.method = "LogNormalize", scale.factor = 10000)
sObject <- FindVariableFeatures(sObject, selection.method = "vst", nfeatures = 2000)
top10 <- head(VariableFeatures(sObject), 10)

# Plot variable features
plot1 <- VariableFeaturePlot(sObject)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot1 + plot2

# Scale data and run PCA
all.genes <- rownames(sObject)
sObject <- ScaleData(sObject, features = all.genes)
sObject <- RunPCA(sObject, features = VariableFeatures(object = sObject))
print(sObject[["pca"]], dims = 1:5, nfeatures = 5)
VizDimLoadings(sObject, dims = 1:2, reduction = "pca")
DimPlot(sObject, reduction = "pca")

# Visualize PCA heatmaps and cluster cells
DimHeatmap(sObject, dims = 1, cells = 500, balanced = TRUE)
DimHeatmap(sObject, dims = 1:15, cells = 500, balanced = TRUE)
sObject <- FindNeighbors(sObject, dims = 1:5)
sObject <- FindClusters(sObject, resolution = 2)
head(Idents(sObject), 5)

# Run UMAP for visualization
sObject <- RunUMAP(sObject, dims = 1:10)
DimPlot(sObject, reduction = "umap")

# Add cell state labels and plot
sObject[["cell_names"]] <- condition_vector
DimPlot(sObject, group.by = "cell_names") + NoLegend()

In [ ]:
# Load custom Seurat utility functions from GitHub
source("https://raw.githubusercontent.com/mjoppich/FlowSets/main/seurat_util_functions.R")

# Save the simulated Seurat object for downstream analysis
saveRDS(sObject, file = "simulated_scdata_random.rds")

# Summarize simulated data by cell state and save to file
summarised_data = getExtendedExpressionData(sObject, group.by="cell_names")
write.table(summarised_data, file = "summarised_simulated_scdata_random.tsv", sep="\t", row.names = FALSE, col.names=TRUE, quote = FALSE)

# Save matrix of genes with changing intensities
write.table(intensity_matrix[unlist(apply(intensity_matrix,1, sd, na.rm = TRUE))!=0, ], file = "simulated_changingGenes_random.tsv", sep="\t", row.names = TRUE, col.names=TRUE, quote = FALSE)


In [ ]:
# Show the list of scaling trends used for pathway changes
trends

In [ ]:
# Plot expression trends for selected genes in each state
options(repr.plot.width = 8, repr.plot.height = 4)

for (stateindex in 1:length(states))
{
    state = names(state2genes)[stateindex]
    trend = trends[stateindex]
    plotnamesuffix = paste(state, paste(trend, sep="->"))

    for (gene in head(state2genes[[state]]))
    {
        gene = make.names(gene)
        p=VlnPlot(sObject, gene, group.by="cell_names") + ggtitle(paste(gene, plotnamesuffix))
        plot(p)

    }

}